In [6]:
from selenium import webdriver
import requests
import time
from sanitize_filename import sanitize
import os
import pickle

In [ ]:
def videoLinks(sidebar):
    collection = []
    # video links are in the side bar
    for i, e in enumerate(sidebar):
        link = e.find_element_by_tag_name('a').get_attribute('href')
        # the title of the video is the title of the class module
        name = e.find_element_by_class_name('classroom-toc-item__title').text
        # the quiz modules are skipped
        if link.find('quiz') > 0:
            pass
        else:
            # the [title, link] is added as list to the collection
            collection.append([str(i) + ' - ' + sanitize(name), link])
    # the [title, link] pairs are returned for the next step
    return collection

def downloadAllVideos(driver, link):
    driver.get(link)
    # get course title
    title = driver.find_element_by_tag_name('h1')
    title = sanitize(title.text)
    # make a directory with the course title if one does not exist yet
    if not os.path.isdir(title):
        os.mkdir(title)
    # get video sidebar
    sidebar = driver.find_elements_by_class_name('classroom-toc-item')
    
    pairs = videoLinks(sidebar)
    for p in pairs:
        time.sleep(5)
        downloadVideo(driver, p, title)
    print(title + 'complete')

def downloadVideo(driver, pair, folder):
    # destructure the variables we need
    [name, link] = pair
    # go to the video page
    driver.get(link)
    # wait 5 seconds the for the page to load
    # TODO: randomize
    time.sleep(5)
    # grab the video source
    videoFrame = driver.find_element_by_tag_name('video').get_attribute('src')
    # the video can be grabbed with a bare request
    s = requests.Session()
    response = s.get(videoFrame, stream=True)
    # 200 = good
    if response.status_code == 200:
        filename = folder + '/' + name +'.mp4'
        with open( filename, 'wb') as f:
            f.write(response.content)
        print(filename + ' successfully retrieved')
        

1. start the browser, go to linkedin learn and log into your account

In [13]:
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/learning/')

1a. save or retrieve cookie from a previous session

In [14]:
# save cookie
pickle.dump( driver.get_cookies() , open("cookies.pkl","wb"))

In [12]:
# retrieve saved cookie
cookies = pickle.load(open("cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

2. go to the the couse

In [24]:
link = 'https://www.linkedin.com/learning/solidworks-sheet-metal-design-2/'

In [25]:
downloadAllVideos(driver, link)

SOLIDWORKS Sheet Metal Design/0 - Getting ahead with sheet metal design in SOLIDWORKS(In progress).mp4 successfully retrieved
SOLIDWORKS Sheet Metal Design/1 - Using the exercise files.mp4 successfully retrieved
SOLIDWORKS Sheet Metal Design/2 - Looking at the sheet metal tools.mp4 successfully retrieved
SOLIDWORKS Sheet Metal Design/3 - Using and customizing the ribbon.mp4 successfully retrieved
SOLIDWORKS Sheet Metal Design/4 - Understanding sheet metal.mp4 successfully retrieved
SOLIDWORKS Sheet Metal Design/5 - Bonus Laser cutting.mp4 successfully retrieved
SOLIDWORKS Sheet Metal Design/6 - Creating a base feature.mp4 successfully retrieved
SOLIDWORKS Sheet Metal Design/7 - Looking at the Flange tool.mp4 successfully retrieved
SOLIDWORKS Sheet Metal Design/8 - Creating tabs.mp4 successfully retrieved
SOLIDWORKS Sheet Metal Design/9 - Making an edge flange.mp4 successfully retrieved
SOLIDWORKS Sheet Metal Design/10 - Using the Edit Flange Profile tool.mp4 successfully retrieved
SOLI